In [1]:
import random

import pandas as pd
import numpy as np
from pybnesian import hc, CLGNetworkType, SemiparametricBNType, SemiparametricBN, MMHC, RCoT, MutualInformation, KMutualInformation, OperatorSet, Score, OperatorPool, ChangeNodeTypeSet, ArcOperatorSet, HoldoutLikelihood, BIC, BGe, CVLikelihood, CKDE, HCKDE, ValidatedLikelihood
#from drawdata import draw_scatter
import matplotlib.pyplot as plt

from bayesace.utils import *
from bayesace.algorithms.bayesace_algortihm import BayesACE
from bayesace.algorithms.face import FACE

from sklearn.preprocessing import StandardScaler
import openml as oml

import time


In [2]:
df = oml.datasets.get_dataset(44091).get_data()[0]


/tmp/ipykernel_377621/1030542227.py:1: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  df = oml.datasets.get_dataset(44091).get_data()[0]


In [3]:
df["class"] = df["quality"].astype('string').astype('category')
df = df.drop("quality", axis=1)

In [4]:
naive = SemiparametricBN(df.columns)
feature_columns = [i for i in df.columns if i != "class"]
df[feature_columns] = StandardScaler().fit_transform(df[feature_columns].values)

In [5]:
for i in [i for i in df.columns if i != "class"] :
    naive.add_arc("class", i)

In [6]:
df_train = df.head(2000)
df_test = df.tail(500)

In [8]:
test = MutualInformation(df_train, True)
#df = pd.read_csv("toy-3class.csv")
#df["class"] = df["z"].astype('category')
#df["z"] = df["z"].astype('category')
#df = df.drop("z", axis = 1 )





learned = hc(df_train, bn_type = CLGNetworkType(), operators = ["arcs"], score = "bic")
learned.fit(df_train)

est = MMHC()
#learned_kde = MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = CVLikelihood(df_train), bn_type = SemiparametricBNType()) #, score = "cv-lik"
#learned_kde = hc(df_train, start = naive, operators = ["node_type","arcs"], score = "holdout-lik", num_folds = 10)
#learned_kde.fit(df_train)



In [347]:
for i in df.columns :
    print(i)
    print(learned_kde.cpd(i))
    print()

fixed acidity
[CKDE] P(fixed acidity | chlorides, residual sugar, alcohol, sulphates, total sulfur dioxide) = CKDE with 2000 instances

volatile acidity
[HCKDE] P(volatile acidity | chlorides, sulphates, pH, class, alcohol, density, total sulfur dioxide)
+-------+--------------------------------------------------------------------------------------------------------------------------+
|       |                   volatile acidity | chlorides, sulphates, pH, alcohol, density, total sulfur dioxide                    |
+-------+--------------------------------------------------------------------------------------------------------------------------+
| class |                                                                                                                          |
+-------+--------------------------------------------------------------------------------------------------------------------------+
| False | [CKDE] P(volatile acidity | chlorides, sulphates, pH, alcohol, density

In [348]:
len([i for i in learned_kde.arcs() if "class" in i])

5

In [71]:
n_exps = 20
df = df.sample(frac = 1)
df_train = df.head(2000)
for i in df_train.columns[:-2] :
    df_train = df_train[df_train[i] <4]
    df_train = df_train[df_train[i] >-4]
df_test = df.tail(500)
for i in df_test.columns[:-2] :
    df_test = df_test[df_test[i] <4]
    df_test = df_test[df_test[i] >-4]

labels = np.zeros(shape = (len(df_test.index)))
labels[df_test["class"] == "True"] = 1

sum_sum = []
accuracy = []
ll = []
num_arcs = []
class_arcs = []
times = []
for i in range(n_exps) :
    t0 = time.process_time()
    learned_kde = hc(df_train, start = naive, operators = ["node_type","arcs"], score = "validated-lik", num_folds = 10)
    #MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = CVLikelihood(df_train), bn_type = SemiparametricBNType())
    learned_kde.fit(df_train)
    times.append(time.process_time()-t0)
    saved = predict_class(df_test, learned_kde)
    accuracy.append(np.sum((labels-saved["True"])**2) / len(df_test.index))
    ll.append(learned_kde.slogl(df_train)/500)
    num_arcs.append(learned_kde.num_arcs())
    class_arcs.append(len([j for j in learned_kde.arcs() if "class" in j]))
    sum = 0
    for j in learned_kde.nodes()[:-2] :    
        if isinstance(learned_kde.cpd(j), CKDE) or isinstance(learned_kde.cpd(j), HCKDE) :
            sum = sum + 1
    sum_sum.append(sum/10*100)
print("Hill climbing with val-lik")
print("MSE:",round(np.mean(accuracy),2),"+-",round(np.std(accuracy),2))
print("LL:",round(np.mean(ll),2),"+-",round(np.std(ll),2))
print("Num arcs:",round(np.mean(num_arcs),2),"+-",round(np.std(num_arcs),2))
print("Class arcs:",round(np.mean(class_arcs),2),"+-",round(np.std(class_arcs),2))
print("KDE%:",round(np.mean(sum_sum),2),"+-",round(np.std(sum_sum),2))
print("Time:",round(np.mean(times),2),"+-",round(np.std(times),2))
analysis_hc = pd.DataFrame(data = np.vstack([accuracy,ll,num_arcs,class_arcs,sum_sum,times]).transpose(), columns = ["MSE", "logl", "n_arcs", "class_arcs","kdes", "ptime"])
print("--------")

sum_sum = []
accuracy = []
ll = []
num_arcs = []
class_arcs = []
times = []
for i in range(n_exps) :
    t0 = time.process_time()
    #learned_kde = hc(df_train, start = naive, operators = ["node_type","arcs"], score = "validated-lik", num_folds = 10)
    learned_kde = MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = ValidatedLikelihood(df_train), bn_type = SemiparametricBNType())
    learned_kde.fit(df_train)
    times.append(time.process_time()-t0)
    saved = predict_class(df_test, learned_kde)
    accuracy.append(np.sum((labels-saved["True"])**2) / len(df_test.index))
    ll.append(learned_kde.slogl(df_train)/500)
    num_arcs.append(learned_kde.num_arcs())
    class_arcs.append(len([j for j in learned_kde.arcs() if "class" in j]))
    sum = 0
    for j in learned_kde.nodes()[:-2] :    
        if isinstance(learned_kde.cpd(j), CKDE) or isinstance(learned_kde.cpd(j), HCKDE) :
            sum = sum + 1
    sum_sum.append(sum/10*100)
print("MMHC with val-lik")
print("MSE:",round(np.mean(accuracy),2),"+-",round(np.std(accuracy),2))
print("LL:",round(np.mean(ll),2),"+-",round(np.std(ll),2))
print("Num arcs:",round(np.mean(num_arcs),2),"+-",round(np.std(num_arcs),2))
print("Class arcs:",round(np.mean(class_arcs),2),"+-",round(np.std(class_arcs),2))
print("KDE%:",round(np.mean(sum_sum),2),"+-",round(np.std(sum_sum),2))
print("Time:",round(np.mean(times),2),"+-",round(np.std(times),2))
analysis_mmhc_val = pd.DataFrame(data = np.vstack([accuracy,ll,num_arcs,class_arcs,sum_sum,times]).transpose(), columns = ["MSE", "logl", "n_arcs", "class_arcs","kdes", "ptime"])
print("--------")

sum_sum = []
accuracy = []
ll = []
num_arcs = []
class_arcs = []
times = []
for i in range(n_exps) :
    t0 = time.process_time()
    #learned_kde = hc(df_train, start = naive, operators = ["node_type","arcs"], score = "validated-lik", num_folds = 10)
    learned_kde = MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = CVLikelihood(df_train), bn_type = SemiparametricBNType())
    learned_kde.fit(df_train)
    times.append(time.process_time()-t0)
    saved = predict_class(df_test, learned_kde)
    accuracy.append(np.sum((labels-saved["True"])**2) / len(df_test.index))
    ll.append(learned_kde.slogl(df_train)/500)
    num_arcs.append(learned_kde.num_arcs())
    class_arcs.append(len([j for j in learned_kde.arcs() if "class" in j]))
    sum = 0
    for j in learned_kde.nodes()[:-2] :    
        if isinstance(learned_kde.cpd(j), CKDE) or isinstance(learned_kde.cpd(j), HCKDE) :
            sum = sum + 1
    sum_sum.append(sum/10*100)
print("MMHC with cv-lik")
print("MSE:",round(np.mean(accuracy),2),"+-",round(np.std(accuracy),2))
print("LL:",round(np.mean(ll),2),"+-",round(np.std(ll),2))
print("Num arcs:",round(np.mean(num_arcs),2),"+-",round(np.std(num_arcs),2))
print("Class arcs:",round(np.mean(class_arcs),2),"+-",round(np.std(class_arcs),2))
print("KDE%:",round(np.mean(sum_sum),2),"+-",round(np.std(sum_sum),2))
print("Time:",round(np.mean(times),2),"+-",round(np.std(times),2))
analysis_mmhc_cv = pd.DataFrame(data = np.vstack([accuracy,ll,num_arcs,class_arcs,sum_sum,times]).transpose(), columns = ["MSE", "logl", "n_arcs", "class_arcs","kdes", "ptime"])
print("--------")

Hill climbing with val-lik
MSE: 0.16 +- 0.01
LL: -32.78 +- 4.03
Num arcs: 36.7 +- 6.91
Class arcs: 10.85 +- 0.48
KDE%: 88.5 +- 7.92
Time: 26.52 +- 10.74
--------
MMHC with val-lik
MSE: 0.19 +- 0.01
LL: -37.66 +- 1.87
Num arcs: 23.1 +- 4.09
Class arcs: 4.05 +- 1.86
KDE%: 93.5 +- 9.1
Time: 12.28 +- 2.72
--------
MMHC with cv-lik
MSE: 0.18 +- 0.0
LL: -35.87 +- 0.14
Num arcs: 28.85 +- 0.36
Class arcs: 7.0 +- 0.0
KDE%: 100.0 +- 0.0
Time: 19.91 +- 1.64
--------


In [75]:
analysis_ga_cv

,MSE,logl,n_arcs,class_arcs,kdes,ptime
0,0.193622,-46.211505,53.0,10.0,0.0,1.459491
1,0.192682,-46.204224,56.0,11.0,0.0,1.393674
2,0.193762,-46.185867,61.0,11.0,0.0,2.557138
3,0.194243,-46.203636,57.0,11.0,0.0,1.479339
4,0.193261,-46.211125,55.0,10.0,0.0,1.286369
5,0.191463,-46.189103,57.0,11.0,0.0,1.849517
6,0.192333,-46.206194,53.0,11.0,0.0,1.214592
7,0.193319,-46.189057,59.0,11.0,0.0,3.024686
8,0.194065,-46.192206,60.0,11.0,0.0,1.686018
9,0.191659,-46.193108,55.0,11.0,0.0,1.733318


In [72]:
n_exps = 20
df = df.sample(frac = 1)
df_train = df.head(2000)
for i in df_train.columns[:-2] :
    df_train = df_train[df_train[i] <4]
    df_train = df_train[df_train[i] >-4]
df_test = df.tail(500)

for i in df_test.columns[:-2] :
    df_test = df_test[df_test[i] <4]
    df_test = df_test[df_test[i] >-4]
    

    
labels = np.zeros(shape = (len(df_test.index)))
labels[df_test["class"] == "True"] = 1

sum_sum = []
accuracy = []
ll = []
num_arcs = []
class_arcs = []
times = []
for i in range(n_exps) :
    t0 = time.process_time()
    learned_kde = hc(df_train, bn_type = CLGNetworkType(), operators = ["arcs"], score = "bic", num_folds = 10)
    #MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = CVLikelihood(df_train), bn_type = SemiparametricBNType())
    learned_kde.fit(df_train)
    times.append(time.process_time()-t0)
    saved = predict_class(df_test, learned_kde)
    accuracy.append(np.sum((labels-saved["True"])**2) / len(df_test.index))
    ll.append(learned_kde.slogl(df_train)/500)
    num_arcs.append(learned_kde.num_arcs())
    class_arcs.append(len([j for j in learned_kde.arcs() if "class" in j]))
    sum = 0
    for j in learned_kde.nodes()[:-2] :    
        if isinstance(learned_kde.cpd(j), CKDE) or isinstance(learned_kde.cpd(j), HCKDE) :
            sum = sum + 1
    sum_sum.append(sum/10*100)
print("Hill climbing with BIC")
print("MSE:",round(np.mean(accuracy),2),"+-",round(np.std(accuracy),2))
print("LL:",round(np.mean(ll),2),"+-",round(np.std(ll),2))
print("Num arcs:",round(np.mean(num_arcs),2),"+-",round(np.std(num_arcs),2))
print("Class arcs:",round(np.mean(class_arcs),2),"+-",round(np.std(class_arcs),2))
print("KDE%:",round(np.mean(sum_sum),2),"+-",round(np.std(sum_sum),2))
print("Time:",round(np.mean(times),2),"+-",round(np.std(times),2))
analysis_ga_bic = pd.DataFrame(data = np.vstack([accuracy,ll,num_arcs,class_arcs,sum_sum,times]).transpose(), columns = ["MSE", "logl", "n_arcs", "class_arcs","kdes", "ptime"])
print("--------")


sum_sum = []
accuracy = []
ll = []
num_arcs = []
class_arcs = []
times = []
for i in range(n_exps) :
    t0 = time.process_time()
    learned_kde = hc(df_train, bn_type = CLGNetworkType(), operators = ["arcs"], score = "validated-lik", num_folds = 10)
    #MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = CVLikelihood(df_train), bn_type = SemiparametricBNType())
    learned_kde.fit(df_train)
    times.append(time.process_time()-t0)
    saved = predict_class(df_test, learned_kde)
    accuracy.append(np.sum((labels-saved["True"])**2) / len(df_test.index))
    ll.append(learned_kde.slogl(df_train)/500)
    num_arcs.append(learned_kde.num_arcs())
    class_arcs.append(len([j for j in learned_kde.arcs() if "class" in j]))
    sum = 0
    for j in learned_kde.nodes()[:-2] :    
        if isinstance(learned_kde.cpd(j), CKDE) or isinstance(learned_kde.cpd(j), HCKDE) :
            sum = sum + 1
    sum_sum.append(sum/10*100)
print("Hill climbing with val-lik")
print("MSE:",round(np.mean(accuracy),2),"+-",round(np.std(accuracy),2))
print("LL:",round(np.mean(ll),2),"+-",round(np.std(ll),2))
print("Num arcs:",round(np.mean(num_arcs),2),"+-",round(np.std(num_arcs),2))
print("Class arcs:",round(np.mean(class_arcs),2),"+-",round(np.std(class_arcs),2))
print("KDE%:",round(np.mean(sum_sum),2),"+-",round(np.std(sum_sum),2))
print("Time:",round(np.mean(times),2),"+-",round(np.std(times),2))
analysis_ga_val = pd.DataFrame(data = np.vstack([accuracy,ll,num_arcs,class_arcs,sum_sum,times]).transpose(), columns = ["MSE", "logl", "n_arcs", "class_arcs","kdes", "ptime"])
print("--------")


sum_sum = []
accuracy = []
ll = []
num_arcs = []
class_arcs = []
times = []
for i in range(n_exps) :
    t0 = time.process_time()
    learned_kde = hc(df_train, bn_type = CLGNetworkType(), operators = ["arcs"], score = "cv-lik", num_folds = 10)
    #MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = CVLikelihood(df_train), bn_type = SemiparametricBNType())
    learned_kde.fit(df_train)
    times.append(time.process_time()-t0)
    saved = predict_class(df_test, learned_kde)
    accuracy.append(np.sum((labels-saved["True"])**2) / len(df_test.index))
    ll.append(learned_kde.slogl(df_train)/500)
    num_arcs.append(learned_kde.num_arcs())
    class_arcs.append(len([j for j in learned_kde.arcs() if "class" in j]))
    sum = 0
    for j in learned_kde.nodes()[:-2] :    
        if isinstance(learned_kde.cpd(j), CKDE) or isinstance(learned_kde.cpd(j), HCKDE) :
            sum = sum + 1
    sum_sum.append(sum/10*100)
print("Hill climbing with cv-lik")
print("MSE:",round(np.mean(accuracy),2),"+-",round(np.std(accuracy),2))
print("LL:",round(np.mean(ll),2),"+-",round(np.std(ll),2))
print("Num arcs:",round(np.mean(num_arcs),2),"+-",round(np.std(num_arcs),2))
print("Class arcs:",round(np.mean(class_arcs),2),"+-",round(np.std(class_arcs),2))
print("KDE%:",round(np.mean(sum_sum),2),"+-",round(np.std(sum_sum),2))
print("Time:",round(np.mean(times),2),"+-",round(np.std(times),2))
analysis_ga_cv = pd.DataFrame(data = np.vstack([accuracy,ll,num_arcs,class_arcs,sum_sum,times]).transpose(), columns = ["MSE", "logl", "n_arcs", "class_arcs","kdes", "ptime"])
print("--------")

Hill climbing with BIC
MSE: 0.19 +- 0.0
LL: -46.32 +- 0.0
Num arcs: 50.0 +- 0.0
Class arcs: 9.0 +- 0.0
KDE%: 0.0 +- 0.0
Time: 0.06 +- 0.0
--------
Hill climbing with val-lik
MSE: 0.2 +- 0.0
LL: -47.1 +- 0.49
Num arcs: 32.7 +- 5.17
Class arcs: 6.0 +- 2.05
KDE%: 0.0 +- 0.0
Time: 0.44 +- 0.11
--------
Hill climbing with cv-lik
MSE: 0.19 +- 0.0
LL: -46.2 +- 0.01
Num arcs: 56.2 +- 2.6
Class arcs: 10.75 +- 0.43
KDE%: 0.0 +- 0.0
Time: 1.77 +- 0.54
--------


In [66]:
from scipy.stats import kstest
from scipy.stats import norm

for i in df.columns[:-2] : 
    print(i,kstest(df[i], 'norm'))


fixed acidity KstestResult(statistic=0.14433382926856342, pvalue=6.806213913198701e-47, statistic_location=0.1729773584075554, statistic_sign=1)
volatile acidity KstestResult(statistic=0.1427379704356445, pvalue=7.228441082937421e-46, statistic_location=0.061327369979953424, statistic_sign=1)
citric acid KstestResult(statistic=0.09933197373690558, pvalue=2.1751231446786863e-22, statistic_location=-0.6365354915635609, statistic_sign=-1)
residual sugar KstestResult(statistic=0.19657333596163662, pvalue=6.010921511702523e-87, statistic_location=-0.5145702936416315, statistic_sign=1)
chlorides KstestResult(statistic=0.1789337559025822, pvalue=5.181167920837191e-72, statistic_location=0.08746373870199499, statistic_sign=1)
free sulfur dioxide KstestResult(statistic=0.05764003476126494, pvalue=8.114116739755302e-08, statistic_location=-1.5715146471652022, statistic_sign=-1)
total sulfur dioxide KstestResult(statistic=0.04586395283077263, pvalue=4.1649165020146406e-05, statistic_location=-1.1

In [52]:
df2 =df2.drop("index",axis=1)

In [35]:
df2 = df.copy()
for i in df2.columns[:-2] :
    df2 = df2[df2[i] <3]
    df2 = df2[df2[i] >-3]

In [36]:
len(df2.index)/len(df.index)

0.9197337509788567

In [372]:
sum = 0
for i in learned_kde.nodes()[:-2] :    
    if isinstance(learned_kde.cpd(i), CKDE) or isinstance(learned_kde.cpd(i), HCKDE) :
        sum = sum + 1
print(sum/10 * 100)

100.0


In [20]:
df_pre = df.copy()

for i in df_pre.columns[:-1] :
    df_pre = df_pre[df_pre[i] <3]
    df_pre = df_pre[df_pre[i] >-3]
    
df_pre = df_pre.reset_index(drop=True)

learned_kde = hc(df_pre, start = naive, operators = ["node_type","arcs"], score = "validated-lik", num_folds = 5)
learned_kde.fit(df_pre)



In [40]:
np.random.seed(0)
t0 = time.process_time()
face = FACE(bayesian_network=learned_kde, features= df_pre.columns[:-1], dataset=df_pre.drop("class", axis = 1).sample(frac = 1).reset_index(drop=True).head(500), graph_type = "epsilon", chunks = 5, penalty=1, distance_threshold=np.sqrt(0.5**2*11), accuracy_threshold= 0.05, likelihood_threshold=0.0005, verbose = True)
res = face.run(df_pre.iloc[[0]])
#print(time.process_time()-t0)

Graph edges:
[(1, 44), (1, 68), (1, 83), (1, 115), (1, 129), (1, 157), (1, 167), (1, 214), (1, 252), (1, 261), (1, 277), (1, 301), (1, 328), (1, 329), (1, 331), (1, 355), (1, 358), (1, 392), (1, 409), (1, 427), (1, 443), (1, 455), (1, 471), (1, 472), (1, 487), (1, 494), (2, 100), (2, 103), (2, 247), (2, 353), (2, 356), (2, 437), (3, 110), (3, 126), (3, 135), (3, 140), (3, 205), (3, 215), (3, 218), (3, 221), (3, 229), (3, 237), (3, 297), (3, 357), (3, 398), (3, 404), (3, 408), (3, 418), (4, 33), (4, 40), (4, 95), (4, 233), (4, 369), (4, 383), (4, 460), (4, 474), (4, 487), (5, 138), (5, 217), (5, 373), (5, 467), (6, 132), (6, 179), (6, 203), (6, 241), (6, 243), (6, 326), (6, 393), (7, 24), (7, 88), (7, 174), (7, 220), (7, 246), (7, 256), (7, 319), (7, 352), (7, 390), (7, 401), (7, 448), (7, 479), (8, 36), (8, 407), (9, 231), (9, 252), (9, 280), (9, 439), (9, 472), (10, 235), (10, 446), (11, 23), (11, 286), (11, 451), (11, 465), (12, 191), (13, 16), (13, 41), (13, 55), (13, 142), (13, 149

In [65]:
path_likelihood_length(res.path,learned_kde)

36.61796837652288

In [64]:
res_opt.distance

array([36.43455058])

In [24]:
t0 = time.time()
n_vertex = 2
x_og = df.iloc[[0]]
bayesace = BayesACE(bayesian_network=learned, features= df.columns[:-1], n_vertex=n_vertex, chunks = 5, penalty=1, pop_size=150, likelihood_threshold=0.0001, accuracy_threshold= 0.1, generations=30)
res_opt = bayesace.run(x_og, n_processes=1)
print(time.time()-t0)

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |      150 |      1 |  0.0000999371 |  0.6452130996 |             - |             -
     2 |      300 |      1 |  0.0000999371 |  0.3260071369 |             - |             -
     3 |      450 |      1 |  0.0000999371 |  0.0296590994 |             - |             -
     4 |      600 |      1 |  0.0000974280 |  0.0000999788 |             - |             -
     5 |      750 |      1 |  0.0000960108 |  0.0000999260 |             - |             -
     6 |      900 |      1 |  0.0000944523 |  0.0000998022 |             - |             -
     7 |     1050 |      1 |  0.0000944523 |  0.0000996579 |             - |             -
     8 |     1200 |      1 |  0.0000936534 |  0.0000993048 |             - |             -
     9 |     1350 |      1 |  0.0000882130 |  0.0000981976 |             - |             -
    10 |     1500 |      1 |  0.0000876779 |  0.0000962414 |             - |             -

In [48]:
res.counterfactual

fixed acidity          -0.728087
volatile acidity        0.260181
citric acid            -0.036504
residual sugar         -0.269755
chlorides              -0.499612
free sulfur dioxide     0.755435
total sulfur dioxide   -0.403147
density                -1.523288
pH                     -0.132385
sulphates              -0.211995
alcohol                 1.904296
Name: 155, dtype: float64

In [25]:
res_opt.counterfactual

fixed acidity           0.817333
volatile acidity       -0.250464
citric acid             0.687118
residual sugar         -0.661246
chlorides               0.113358
free sulfur dioxide    -1.014017
total sulfur dioxide   -1.213462
density                -0.550290
pH                      0.389414
sulphates               0.293936
alcohol                 1.043154
Name: 3, dtype: float64

In [26]:
res_opt.counterfactual

fixed acidity           0.817333
volatile acidity       -0.250464
citric acid             0.687118
residual sugar         -0.661246
chlorides               0.113358
free sulfur dioxide    -1.014017
total sulfur dioxide   -1.213462
density                -0.550290
pH                      0.389414
sulphates               0.293936
alcohol                 1.043154
Name: 3, dtype: float64

In [19]:
np.sum(np.abs(res.counterfactual-res_opt.counterfactual))/11/6

NameError: name 'res' is not defined

In [51]:
res_opt.path

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,1.074041,-0.071241,-0.186512,-0.959689,-0.206074,-0.833701,-0.088889,-0.506999,-1.707929,-0.930081,-0.581331
1,-0.782993,0.417737,-0.583686,0.333003,-0.549122,0.051426,0.404042,-0.705472,0.485267,-0.434907,0.748370
2,-0.611261,0.061735,-0.322317,-0.580877,-0.882620,0.770653,0.020193,-1.540727,0.239687,-0.330372,1.943428


In [169]:
res.path

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
1454,-1.554062,1.254447,-1.086559,-0.692618,-0.30392,-0.833701,-0.236776,-0.536222,1.50618,2.203383,1.593592


In [170]:
df.iloc[[0]]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,class
589,3.551967,0.39275,1.238564,-0.559082,1.522538,-1.060721,-1.327436,2.061323,-0.384472,0.50609,-0.425979,False


In [171]:
df.loc[[1454]]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,class
1454,-1.554062,1.254447,-1.086559,-0.692618,-0.30392,-0.833701,-0.236776,-0.536222,1.50618,2.203383,1.593592,True


In [189]:
res.path

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
1454,-1.554062,1.254447,-1.086559,-0.692618,-0.303920,-0.833701,-0.236776,-0.536222,1.506180,2.203383,1.593592
1822,-1.178619,1.121878,-1.461579,-0.158475,-0.401766,-0.776946,0.040511,-1.088200,1.254093,2.399224,2.214999


In [190]:
df.iloc[0]

fixed acidity           3.551967
volatile acidity         0.39275
citric acid             1.238564
residual sugar         -0.559082
chlorides               1.522538
free sulfur dioxide    -1.060721
total sulfur dioxide   -1.327436
density                 2.061323
pH                     -0.384472
sulphates                0.50609
alcohol                -0.425979
class                      False
Name: 589, dtype: object

In [191]:
res.counterfactual

fixed acidity          -1.178619
volatile acidity        1.121878
citric acid            -1.461579
residual sugar         -0.158475
chlorides              -0.401766
free sulfur dioxide    -0.776946
total sulfur dioxide    0.040511
density                -1.088200
pH                      1.254093
sulphates               2.399224
alcohol                 2.214999
Name: 1822, dtype: float64